In [5]:
import os
import numpy as np
import math
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch

In [6]:
epochs, batch_size  = 100, 64
lr, b1, b2 = 2e-4, 0.5, 0.999
latent_dim = 100
img_size = 28
channels = 1
img_shape = (channels, img_size, img_size)
if torch.cuda.is_available(): 
    print("Train on GPU \nCUDA is available")
    cuda = True 
else:
    print("Train on the CPU \nCUDA is not available")
    cuda = False


Train on GPU 
CUDA is available


In [7]:
os.makedirs("data/mnist", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose([
            transforms.Resize(img_size),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=batch_size,
    shuffle=True,
)

In [8]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.shape[0], *img_shape)
        return img

In [9]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, img):
        flat_img = img.view(img.size(0), -1)
        pred = self.model(flat_img)
        return pred

In [10]:
G = Generator()
D = Discriminator()
adversarial_loss = torch.nn.BCELoss()

if cuda:
    G.cuda()
    D.cuda()
    adversarial_loss.cuda()

In [10]:
optimizer_G = torch.optim.Adam(G.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(D.parameters(), lr=lr, betas=(b1, b2))
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
os.makedirs("GAN_results", exist_ok=True)

for epoch in range(epochs):
    for i, (imgs, _) in enumerate(dataloader):
        real = Variable(Tensor(imgs.size(0), 1).fill_(1.0),
                        requires_grad=False)
        fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0),
                        requires_grad=False)

        real_imgs = Variable(imgs.type(Tensor))

        ## Train Generator ##
        optimizer_G.zero_grad()
        z = Variable(Tensor(np.random.normal(0, 1,
                                             (imgs.shape[0], latent_dim))))
        gen_imgs = G(z)
        g_loss = adversarial_loss(D(gen_imgs), real)
        g_loss.backward()
        optimizer_G.step()

        ## Train Discriminator ##
        optimizer_D.zero_grad()
        real_loss = adversarial_loss(D(real_imgs), real)
        fake_loss = adversarial_loss(D(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()

    print("[Epoch %d/%d] [D loss: %f] [G loss: %f]"
          % ((epoch+1), epochs, d_loss.item(), g_loss.item()))
    save_image(gen_imgs.data[:25], "GAN_results/epoch_%d.png"
               %epoch, nrow=5, normalize=True)

torch.save(G.state_dict(), './Generator.pth')
torch.save(D.state_dict(), './Discriminator.pth')

[Epoch 0/2] [D loss: 0.469341] [G loss: 1.281611]
[Epoch 1/2] [D loss: 0.563897] [G loss: 1.102454]
